Copyright **`(c)`** 2023 Antonio Ferrigno `<s316467@polito.it>`  
[`https://github.com/s316467/Computational-Intelligence-23-24/tree/main`](https://github.com/s316467/Computational-Intelligence-23-24/tree/main)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/s316467/Computational-Intelligence-23-24/tree/main/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [19]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [20]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [21]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [22]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [23]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [24]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


## Evolutional Strategy approach

In [25]:
number_of_games = 100  # define the number of games each agent should play to evaluate its fitness

perfect_score = number_of_games # define the perfect score condition

class NimAgent:
    def __init__(self, genome):
        self.genome = genome

    def make_move(self, state):
        # Here we'll just use a simple strategy based on the genome.
        # Let's assume the genome is a list of preferences for each row.
        # The higher the number, the more the agent prefers to take from that row.
        
        # Get the non-empty rows and their respective preferences from the genome
        moves = [(row, min(state.rows[row], max(1, int(self.genome[row] * state.rows[row]))))
                 for row in range(len(state.rows)) if state.rows[row] > 0]

        # Select the move with the highest preference
        # (You could also incorporate randomness here based on the genome)
        chosen_move = max(moves, key=lambda x: self.genome[x[0]])

        # Return the move as a Nimply object
        return Nimply(*chosen_move)

def play_game(agent, nim):
    # Logic to play a game of Nim using the agent's strategy
    while nim:
        ply = agent.make_move(nim)
        nim.nimming(ply)
        if not nim:
            return 1  # agent wins
    return 0  # agent loses

def crossover(genome1, genome2):
    # Logic for combining two genomes
    child_genome = []
    for g1, g2 in zip(genome1, genome2):
        child_genome.append(random.choice([g1, g2]))
    return child_genome

def mutate(genome, mutation_rate):
    # Logic for mutating a genome
    for i in range(len(genome)):
        if random.random() < mutation_rate:
            genome[i] += random.choice([-1, 1]) * random.random()  # Adjust mutation as per requirement
    return genome

def select_agents(population, fitness_scores):
    # Logic to select the fittest agents
    # Let's use a simple tournament selection
    selected = []
    while len(selected) < len(population) // 2:
        participant = random.sample(list(zip(population, fitness_scores)), 2)
        winner = max(participant, key=lambda x: x[1])
        selected.append(winner[0])
    return selected


def initialize_population(pop_size, genome_length):
    population = [NimAgent([random.random() for _ in range(genome_length)]) for _ in range(pop_size)]
    return population

def evaluate_population(population, nim):
    fitness_scores = []

    for agent in population:
        score = 0
        # Play a number of games and record the score
        for _ in range(number_of_games):
            result = play_game(agent, nim)
            score += result
        fitness_scores.append(score)
    
    return fitness_scores

def reproduce(selected_agents, mutation_rate):
    new_population = []

    while len(new_population) < len(selected_agents):
        parent1, parent2 = random.sample(selected_agents, 2)
        child_genome = crossover(parent1.genome, parent2.genome)
        child_genome = mutate(child_genome, mutation_rate)
        new_population.append(NimAgent(child_genome))
    
    return new_population

def replacement(population, new_population, fitness_scores):
    # Sort the population by fitness and replace the worst ones
    sorted_population = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)
    survivors = sorted_population[:len(population) - len(new_population)]
    return [agent for agent, score in survivors] + new_population

def evolutionary_strategy(nim, generations, pop_size, mutation_rate):
    population = initialize_population(pop_size, len(nim.rows))

    for generation in range(generations):
        fitness_scores = evaluate_population(population, nim)
        selected_agents = select_agents(population, fitness_scores)
        new_population = reproduce(selected_agents, mutation_rate)
        population = replacement(population, new_population, fitness_scores)
        print(f"Generation {generation}: Best score {max(fitness_scores)}")

        # Check for termination condition (e.g., a perfect score)
        if max(fitness_scores) >= perfect_score:
            break

    return population


In [26]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [38]:
import logging

# Set the logging level to INFO to see the details of the game play
logging.getLogger().setLevel(logging.INFO)

# Set parameters for the evolutionary strategy
num_rows = 5  # number of rows in the Nim game
generations = 50  # number of generations to run the evolution
pop_size = 100  # size of the agent population
mutation_rate = 0.05  # mutation rate for the evolutionary strategy

# Initialize the Nim game
nim_game = Nim(num_rows)

# Run the evolutionary strategy to get a population of evolved agents
evolved_population = evolutionary_strategy(nim_game, generations, pop_size, mutation_rate)

# Evaluate the final population to find the best agent
fitness_scores = evaluate_population(evolved_population, nim_game)
best_agent_index = fitness_scores.index(max(fitness_scores))
best_agent = evolved_population[best_agent_index]

# Now let's play a game with the best agent from the evolutionary strategy
nim = Nim(num_rows)
player = 0  # Let's say our evolved agent is player 0

# Select a strategy for the opponent, for example, a pure random strategy
opponent_strategy = pure_random

logging.info(f"Initial state: {nim}")

while nim:
    if player == 0:
        # Use the best agent found by the evolutionary strategy to play
        ply = best_agent.make_move(nim)
    else:
        # Use a pre-defined strategy for the opponent
        ply = opponent_strategy(nim)

    logging.info(f"Player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"State after move: {nim}")

    if not nim:
        logging.info(f"Player {player} wins!")
        break

    player = 1 - player  # Switch players


INFO:root:Initial state: <1 3 5 7 9>
INFO:root:Player 0 plays Nimply(row=1, num_objects=2)
INFO:root:State after move: <1 1 5 7 9>
INFO:root:Player 1 plays Nimply(row=1, num_objects=1)
INFO:root:State after move: <1 0 5 7 9>
INFO:root:Player 0 plays Nimply(row=0, num_objects=1)
INFO:root:State after move: <0 0 5 7 9>
INFO:root:Player 1 plays Nimply(row=2, num_objects=2)
INFO:root:State after move: <0 0 3 7 9>
INFO:root:Player 0 plays Nimply(row=2, num_objects=2)
INFO:root:State after move: <0 0 1 7 9>
INFO:root:Player 1 plays Nimply(row=3, num_objects=4)
INFO:root:State after move: <0 0 1 3 9>
INFO:root:Player 0 plays Nimply(row=2, num_objects=1)
INFO:root:State after move: <0 0 0 3 9>
INFO:root:Player 1 plays Nimply(row=4, num_objects=9)
INFO:root:State after move: <0 0 0 3 0>
INFO:root:Player 0 plays Nimply(row=3, num_objects=1)
INFO:root:State after move: <0 0 0 2 0>
INFO:root:Player 1 plays Nimply(row=3, num_objects=2)
INFO:root:State after move: <0 0 0 0 0>
INFO:root:Player 1 wins

Generation 0: Best score 1
Generation 1: Best score 0
Generation 2: Best score 0
Generation 3: Best score 0
Generation 4: Best score 0
Generation 5: Best score 0
Generation 6: Best score 0
Generation 7: Best score 0
Generation 8: Best score 0
Generation 9: Best score 0
Generation 10: Best score 0
Generation 11: Best score 0
Generation 12: Best score 0
Generation 13: Best score 0
Generation 14: Best score 0
Generation 15: Best score 0
Generation 16: Best score 0
Generation 17: Best score 0
Generation 18: Best score 0
Generation 19: Best score 0
Generation 20: Best score 0
Generation 21: Best score 0
Generation 22: Best score 0
Generation 23: Best score 0
Generation 24: Best score 0
Generation 25: Best score 0
Generation 26: Best score 0
Generation 27: Best score 0
Generation 28: Best score 0
Generation 29: Best score 0
Generation 30: Best score 0
Generation 31: Best score 0
Generation 32: Best score 0
Generation 33: Best score 0
Generation 34: Best score 0
Generation 35: Best score 0
Ge